# Hyperparameter Tuning with Optuna
Optimize XGBoost hyperparameters using Bayesian optimization

In [ ]:
import optuna
from optuna.pruners import MedianPruner
import xgboost as xgb
from sklearn.model_selection import cross_val_score
import numpy as np

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }
    
    model = xgb.XGBRegressor(**params, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    return scores.mean()

study = optuna.create_study(direction='maximize', pruner=MedianPruner())
study.optimize(objective, n_trials=100)

print(f'Best R² Score: {study.best_value:.4f}')
print(f'Best Parameters: {study.best_params}')

In [ ]:
# Visualize optimization history
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
trial_values = [trial.value for trial in study.trials if trial.value is not None]
plt.plot(trial_values, marker='o', alpha=0.6)
plt.xlabel('Trial')
plt.ylabel('R² Score')
plt.title('Optuna Optimization History')
plt.grid(True)
plt.show()

# Parameter importance
fig = optuna.visualization.plot_param_importances(study).to_dict()